In [1]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from typing import Iterable, List
from translation.translation_dataloader import TranslationDataset
from translation.models.transfer_learning_transformer.gec_dataloader import GecDataset
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
from translation.models.baseline_transformer.model import Seq2SeqTransformer
import nltk
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(DEVICE)

cuda


In [2]:
def build_text_transform():
    def sequential_transforms(*transforms):
        def func(txt_input):
            for transform in transforms:
                txt_input = transform(txt_input)
            return txt_input
        return func

    def tensor_transform(token_ids):
        return torch.cat((torch.tensor([BOS_IDX]),
                          torch.tensor(token_ids),
                          torch.tensor([EOS_IDX])))

    text_transform = sequential_transforms(token_transform,
                                               vocab_transform,
                                               tensor_transform)
    return text_transform

def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len), device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

def collate_fn(batch):
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        src_batch.append(build_text_transform()(src_sample.rstrip("\n")))
        tgt_batch.append(build_text_transform()(tgt_sample.rstrip("\n")))

    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
    return src_batch, tgt_batch

def translate(model: torch.nn.Module, src_sentence: str):
    model.eval()
    src = build_text_transform()(src_sentence).view(-1, 1)
    num_tokens = src.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(
        model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()
    return " ".join(vocab_transform.lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<bos>", "").replace("<eos>", "")

def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len-1):
        memory = memory.to(DEVICE)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                    .type(torch.bool)).to(DEVICE)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
            break
    return ys

def bleu_evaluation(transformer, test_iter):
    score = 0
    for sample, target in test_iter:
        prediction = translate(transformer, sample).split()
        print(sample, "-", " ".join(prediction))
        target = target.split()
        if len(sample) < 3 or len(target) < 3:
            BLEUscore = nltk.translate.bleu_score.sentence_bleu([target], prediction, weights=[0.5, 0.5, 0, 0])
        else:
            BLEUscore = nltk.translate.bleu_score.sentence_bleu([target], prediction)
        score += BLEUscore
    return score / len(test_iter)


In [3]:
data = TranslationDataset(("E:/Python/Projects/PSL-Translator/translation/data_collection/data.txt",))
data_gec = GecDataset("gec_dataset.jsonl")

In [4]:
token_transform = get_tokenizer('spacy', language='pl_core_news_sm')

In [5]:
def yield_tokens(data_iter: Iterable) -> List[str]:
    for data_sample in data_iter:
        yield token_transform(data_sample[0])
        yield token_transform(data_sample[1])

In [6]:
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

In [7]:
vocab_transform = build_vocab_from_iterator(yield_tokens(data),
                                                    min_freq=1,
                                                    specials=special_symbols,
                                                    special_first=True)
vocab_transform.set_default_index(UNK_IDX)

In [8]:
torch.manual_seed(0)

VOCAB_SIZE = len(vocab_transform)
EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 512
BATCH_SIZE = 128
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3

train_iter, val_iter, test_iter = torch.utils.data.random_split(data, [0.9, 0.05, 0.05])
train_dataloader = DataLoader(train_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)
val_dataloader = DataLoader(val_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)

gec_train_iter, gec_val_iter = torch.utils.data.random_split(data_gec, [0.97, 0.03])
gec_train_dataloader = DataLoader(gec_train_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)
gec_val_dataloader = DataLoader(gec_val_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)

transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, VOCAB_SIZE, VOCAB_SIZE, FFN_HID_DIM)

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(DEVICE)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

E:\Python\Projects\pythonProject4\venv\lib\site-packages\torch\nn\modules\transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [9]:

def train_epoch(model, optimizer, dataloader):
    model.train()
    losses = 0
    
    
    

    for src, tgt in dataloader:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        optimizer.zero_grad()

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()

        optimizer.step()
        losses += loss.item()

    return losses / len(list(dataloader))


def evaluate(model, dataloader):
    model.eval()
    losses = 0

    

    for src, tgt in dataloader:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        losses += loss.item()

    return losses / len(list(dataloader))

In [10]:
from timeit import default_timer as timer
NUM_EPOCHS_GEC = 10


for epoch in range(1, NUM_EPOCHS_GEC+1):
    start_time = timer()
    train_loss = train_epoch(transformer, optimizer, gec_train_dataloader)
    end_time = timer()
    val_loss = evaluate(transformer, gec_val_dataloader)
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))
    
torch.save(transformer.state_dict(), "gec_transformer.pth")

E:\Python\Projects\pythonProject4\venv\lib\site-packages\torch\nn\functional.py:5476: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)
E:\Python\Projects\pythonProject4\venv\lib\site-packages\torch\nn\functional.py:5109: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


Epoch: 1, Train loss: 2.385, Val loss: 1.834, Epoch time = 109.996s
Epoch: 2, Train loss: 1.664, Val loss: 1.549, Epoch time = 114.957s
Epoch: 3, Train loss: 1.395, Val loss: 1.246, Epoch time = 116.257s
Epoch: 4, Train loss: 1.150, Val loss: 1.021, Epoch time = 117.697s
Epoch: 5, Train loss: 0.942, Val loss: 0.806, Epoch time = 117.929s
Epoch: 6, Train loss: 0.790, Val loss: 0.681, Epoch time = 118.334s
Epoch: 7, Train loss: 0.671, Val loss: 0.555, Epoch time = 121.415s
Epoch: 8, Train loss: 0.586, Val loss: 0.491, Epoch time = 123.742s
Epoch: 9, Train loss: 0.520, Val loss: 0.448, Epoch time = 124.228s
Epoch: 10, Train loss: 0.465, Val loss: 0.437, Epoch time = 123.470s


In [15]:
NUM_EPOCHS_TRANSLATION = 20
transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, VOCAB_SIZE, VOCAB_SIZE, FFN_HID_DIM)
transformer.to(DEVICE)
transformer.load_state_dict(torch.load('gec_transformer.pth'))
optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)
transformer.train()

for epoch in range(1, NUM_EPOCHS_TRANSLATION+1):
    start_time = timer()
    train_loss = train_epoch(transformer, optimizer, train_dataloader)
    end_time = timer()
    val_loss = evaluate(transformer, val_dataloader)
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))
    
torch.save(transformer.state_dict(), "pretrained_transformer.pth")

Epoch: 1, Train loss: 5.130, Val loss: 4.754, Epoch time = 0.975s
Epoch: 2, Train loss: 4.450, Val loss: 4.386, Epoch time = 0.772s
Epoch: 3, Train loss: 4.128, Val loss: 4.246, Epoch time = 0.752s
Epoch: 4, Train loss: 3.846, Val loss: 4.156, Epoch time = 0.753s
Epoch: 5, Train loss: 3.653, Val loss: 4.043, Epoch time = 0.749s
Epoch: 6, Train loss: 3.426, Val loss: 3.918, Epoch time = 0.758s
Epoch: 7, Train loss: 3.266, Val loss: 3.837, Epoch time = 0.755s
Epoch: 8, Train loss: 3.114, Val loss: 3.836, Epoch time = 0.759s
Epoch: 9, Train loss: 3.004, Val loss: 3.803, Epoch time = 0.757s
Epoch: 10, Train loss: 2.922, Val loss: 4.002, Epoch time = 0.761s
Epoch: 11, Train loss: 2.762, Val loss: 3.813, Epoch time = 0.757s
Epoch: 12, Train loss: 2.607, Val loss: 3.670, Epoch time = 0.763s
Epoch: 13, Train loss: 2.477, Val loss: 3.669, Epoch time = 0.755s
Epoch: 14, Train loss: 2.357, Val loss: 3.695, Epoch time = 0.763s
Epoch: 15, Train loss: 2.263, Val loss: 3.664, Epoch time = 0.762s
Epoc

In [12]:
print("Your BLEU score is: " + str(bleu_evaluation(transformer, test_iter)))

Czy my razem podróż samochód chcieć - Czy razem razem razem samochód ?
Ja mama odwiedzić będzie - Ja będzie będzie będzie .
Ty zły czemu - Ty Ty ?
Ja góry zdobywać chcieć - Ja Ja ?
Ty sklep iść? - Ty Ty ?
Ty książka czytać ukończyć jeszcze - Ty jeszcze jeszcze jeszcze ?
Ty rower park jechać byli? - Ty mają jesteś byli byli ?


E:\Python\Projects\pythonProject4\venv\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
E:\Python\Projects\pythonProject4\venv\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
E:\Python\Projects\pythonProject4\venv\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram

On wysoki człowiek być - On ty być być .
Ja się dobrze czuć. - Ja się dobrze .
Ty na mnie zły być teraz? - Ty na mnie być teraz teraz ?
Ja dobrze - Ja dobrze .
Ty rower jeździć? - Ty ?
Ty mi pomożesz? - Ty mi ?
Ja telefon nowy kupić planować - Ja Ja kiedy ciocia ?
Ona poniedziałek wolne mieć - Ona mam mieć ?
On pomóc praca wczoraj byli. - On On , kto byli .
Ja kawa pić zawsze - Ja długo zawsze ?
My razem film oglądać - Ty razem razem .
Ty woda pić? - Ty ?
Jaką muzyka najbardziej lubić - Jest najbardziej najbardziej ?
Ty szukać hotel co? - Ty ? raz co ?
Dziadek na rower jeździć codziennie - Czy na zawsze ?
Księżyc jasny dzisiaj - ?
Kiedy zaczynać ten koncert - Kiedy ten ten ?
On muzyka grać - On On ?
Ty pomóc ja pytanie - Ty ja ja ?
Ty zmęczony wyglądać - Ty ?
Książka na stole leżeć - Myślę na stole ?
On uśmiechać się - On się i ?
Czy ona teatr weekend iść - Czy ona raz ?
Drzewo park rosnąć - Teraz ?
Twoja rodzina muzeum odwiedzać chciała? - Nigdy ty ty ty ?
My tata nowy dom kupić chcie